In [5]:
/*
 * Si Python et/ou R ne fonctionnent pas, voyez: https://github.com/dotnet/interactive/blob/HEAD/docs/jupyter-in-polyglot-notebooks.md
 *
 * Enlevez les commentaires qui suivent afin de connecter Python et R.
 */

// #!connect jupyter --kernel-name pythonkernel --kernel-spec python3

// #!connect jupyter --kernel-name Rkernel --kernel-spec ir

In [28]:
#r "nuget: Microsoft.Extensions.Http, 8.0.0"

Installed Packages Microsoft.Extensions.Http, 8.0.0

## Détermination des dates du premier et dernier circuit

Afin de sélectionner alléatoirement des circuits, il est necéssaire de savoir l'ensemble des circuits possibles. À ce fin, puisque l'API de TMX n'expose aucune méthode de choix alléatoire de circuit, la date de publication de chaque circuit sera utilisée.

In [1]:

public enum Order1 {
  None = 0,
  UploadDateOldest = 1,
  UploadDateNewestDefault = 2,
  UpdateDateOldest = 3,
  UpdateDateNewest = 4,
  AwardCountAscending = 5,
  AwardCountDescending = 6,
  CommentCountAscending = 7,
  CommentCountDescending = 8,
  ActivityOldest = 9,
  ActivityNewest = 10,
  TrackNameAZ = 11,
  TrackNameZA = 12,
  AuthorNameAZ = 13,
  AuthorNameZA = 14,
  DifficultyAscending = 15,
  DifficultyDescending = 16,
  DownloadCountAscending = 17,
  DownloadCountDescending = 18,
  TrackValueAscending = 19,
  TrackValueDescending = 20,
  AwardsthisweekAscending = 21,
  AwardsthisweekDescending = 22,
  AwardsthismonthAscending = 23,
  AwardsthismonthDescending = 24,
  LastAwardDateOldest = 25,
  LastAwardDateNewest = 26,
  WorldRecordsetOldest = 27,
  WorldRecordsetNewest = 28,
  PersonalrecordsetOldestLoginrequired = 29,
  PersonalrecordsetNewestLoginrequired = 30,
  PersonalawardsetOldestLoginrequired = 31,
  PersonalawardsetNewestLoginrequired = 32,
  PersonaldownloadOldestLoginrequired = 33,
  PersonaldownloadNewestLoginrequired = 34,
  PersonalcommentsetOldestLoginrequired = 35,
  PersonalcommentsetNewestLoginrequired = 36,
  UserRecordPositionLowest = 37,
  UserRecordPositionHighest = 38,
}

public enum Tag {
  Normal = 0,
  //Stunt = 1,
  //Maze = 2,
  Offroad = 3,
  //Laps = 4,
  Fullspeed = 5,
  //LOL = 6,
  Tech = 7,
  SpeedTech = 8,
  //RPG = 9,
  PressForward = 10,
  //Trial = 11,
  Grass = 12,
}

public enum PrimaryType {
  Race = 0,
  Puzzle = 1,
  Platform = 2,
  Stunts = 3,
  Shortcut = 4,
  Laps = 5,
}

public enum LBType {
  Allreplaytypes = -1,
  Normal = 0,
  Classic = 1,
  Nadeo = 2,
  Useless = 3,
}

public enum Difficulty {
  Beginner = 0,
  Intermediate = 1,
  Expert = 2,
  Lunatic = 3,
}

public enum Routes {
  Single = 0,
  Multiple = 1,
  Symmetrical = 2,
}

In [2]:
#nullable enable

public class TMXResponses
{
  public static DateTime newestMapAt { get; set; }
  public static DateTime oldestMapAt { get; set; }

  public class ParsedResponse<T>
  {
    public Boolean More { get; set; }
    public T[]? Results { get; set; }
  }

  public class MapResponse
  {
      public int? TrackId { get; set; }
      public string? TrackName { get; set; }
      public string? UId { get; set; }
      public int? AuthorTime { get; set; }
      public Uploader? Uploader { get; set; }
      public DateTime? UploadedAt { get; set; }
      public DateTime? UpdatedAt { get; set; }
      public int? PrimaryType { get; set; } // enum
      public int? TrackValue { get; set; }
      public int? Style { get; set; } // enum
      public int? Routes { get; set; } // enum
      public int? Difficulty { get; set; }
      public int? Environment { get; set; } // enum
      public int? Car { get; set; } // enum
      public WRReplay? WRReplay { get; set; }
  }

  public class Uploader
  {
    public int? UserId { get; set; }
    public string? Name { get; set; }
  }

  public class WRReplay
  {
    public int ReplayId { get; set; }
    public int ReplayTime { get; set; }
  }
}

In [19]:
#nullable enable

public class TMXInputs
{
  public class MapInputsParameters {
      /// <summary>
      /// Comma-seperated and URL-Encoded list of fields to display in the response.
      /// </summary>
    public String[]? fields { get; set; }
      /// <summary>
      /// Search Order
      /// </summary>
    public Order1? order1 { get; set; }
      /// <summary>
      /// Amount of results (default: 40)
      /// </summary>
    public Int32? count { get; set; }
      /// <summary>
      /// Display results after specified TrackId and given order - used for paging
      /// </summary>
    public Int64? after { get; set; }
      /// <summary>
      /// Display results before specified TrackId and given order - used for paging
      /// </summary>
    public Int64? before { get; set; }
      /// <summary>
      /// Like after, but includes the specified TrackId as well
      /// </summary>
    public Int64? from { get; set; }
      /// <summary>
      /// Filter by TrackId's
      /// </summary>
    public Int64[,]? id { get; set; }
      /// <summary>
      /// Filter by UId or Track file signatures
      /// </summary>
    public String[,]? uid { get; set; }
      /// <summary>
      /// Filter by Tags[]
      /// </summary>
    public Tag[]? tag { get; set; }
      /// <summary>
      /// Filter by excluding Tags[]
      /// </summary>
    public Tag[]? etag { get; set; }
      /// <summary>
      /// true: track must have all specified tags, false: track must have one of the specified tags
      /// </summary>
    public Boolean taginclusive { get; } = true;
      /// <summary>
      /// Filter by PrimaryType
      /// </summary>
    public PrimaryType primarytype { get; } = PrimaryType.Race;
      /// <summary>
      /// Filter by min. UploadedAt
      /// </summary>
    public DateTime? uploadedafter { get; set; }
      /// <summary>
      /// Filter by max. UploadedAt
      /// </summary>
    public DateTime? uploadedbefore { get; set; }
      /// <summary>
      /// Filter by min. AuthorTime in ms
      /// </summary>
    public Int32? authortimemin { get; } = 30000;
      /// <summary>
      /// Filter by max. AuthorTime in ms
      /// </summary>
    public Int32? authortimemax { get; } = 120000;
      /// <summary>
      /// Filter by Routes
      /// </summary>
    public Routes routes { get; } = Routes.Single;
      /// <summary>
      /// 1: Track has at least one replay
      /// </summary>
    public Int32 inhasrecord { get; } = 1;
      /// <summary>
      /// 1: Track is environment mixed (Environment =/= Car)
      /// </summary>
    public Int32 inenvmix { get; } = 0;
      /// <summary>
      /// 1: Track requires to be played with TMUnlimiter, 0: vanilla game
      /// </summary>
    public Int32 inunlimiter { get; } = 0;

    public override string ToString(){
      // Voir https://stackoverflow.com/questions/4276566/how-can-you-loop-over-the-properties-of-a-class

      StringBuilder paramaterString = new StringBuilder();

      Boolean needsAnd = false;
      foreach(var prop in this.GetType().GetProperties()) {
        object? value = prop.GetValue(this, null);

        if (value == null) { continue; }

        if (needsAnd) {
          paramaterString.Append("&");
        } else { needsAnd = true; }

        if (prop.Name == "fields") {
          paramaterString.Append($"fields={String.Join("%2C", value)}");
          continue;
        }

        // If DateTime
        if (value is DateTime) {
          string dateString = ((DateTime) value).ToString("s", System.Globalization.CultureInfo.InvariantCulture);

          paramaterString.Append($"{prop.Name}={dateString}");
          continue;
        }

        // If Enum[]
        if (value is Enum[]) {
          paramaterString.Append($"{prop.Name}=");

          var enumValue = (Enum[]) value;

          paramaterString.Append(enumValue[0].ToString("D"));

          for (int i = 1; i < enumValue.Length; i++) {
            paramaterString.Append($"%2C{enumValue[0].ToString("D")}");
          }
          
          continue;
        }

        // If Enum
        if (value is Enum) {
          paramaterString.Append($"{prop.Name}={((Enum) value).ToString("D")}");
          continue;
        }

        paramaterString.Append($"{prop.Name}={value}");
      }
      
      return paramaterString.ToString();
    }  
  }
}

In [29]:
using System;
using System.Globalization;
using System.Net.Http;
using System.Net;
using System.Text.Json;

#nullable enable

public class TMXAPI
{
  public static HttpClient sharedClient = new()
  {
      BaseAddress = new Uri("https://tmnf.exchange"),
  };

  // private static async Task<TMXResponses.ParsedResponse<TMXResponses.MapResponse>?> getMapResponse(TMXInputs.MapInputsParameters paramaters)
  // {
  //   using HttpResponseMessage response = await sharedClient.GetAsync($"api/tracks?{paramaters}"); // Note: If "parameters" can be chosen by the user, add sanitization

  //   Console.WriteLine($"Get: api/tracks?{paramaters}");
    
  //   response.EnsureSuccessStatusCode();
    
  //   var jsonResponse = await response.Content.ReadAsStringAsync();

  //   TMXResponses.ParsedResponse<TMXResponses.MapResponse>? mapResponse = JsonSerializer.Deserialize<TMXResponses.ParsedResponse<TMXResponses.MapResponse>>(jsonResponse);

  //   return mapResponse;
  // }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L200
  private static Uri? GetRequestUriFromResponse(HttpResponseMessage response)
  {
      if (response.RequestMessage is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request message. This is odd...");
          return null;
      }

      if (response.RequestMessage.RequestUri is null)
      {
          Console.Error.WriteLine("Response from the HEAD request does not contain information about the request URI. This is odd...");
          return null;
      }

      return response.RequestMessage.RequestUri;
  }

  // Voir https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs#L217
  private static string? GetTrackIdFromUri(Uri uri)
  {
      var trackId = uri.Segments.LastOrDefault();

      if (trackId is null) { return null; }

      return trackId;
  }

  // Voir: https://github.com/BigBang1112/randomizer-tmf/blob/bae33563fe4116b22be5b099a51a5d311dd3366c/Src/RandomizerTMF.Logic/Services/MapDownloader.cs
  public static async Task<string?> getRandomMapID(TMXInputs.MapInputsParameters paramaters)
  {
    using var response = await sharedClient.SendAsync(new HttpRequestMessage(HttpMethod.Head, $"trackrandom?{paramaters}")); // Note: If "parameters" can be chosen by the user, add sanitization

    Console.WriteLine($"Head: trackrandom?{paramaters}");

    if (response.StatusCode == HttpStatusCode.NotFound)
		{
			// I have written something incorrectly

			throw new Exception("No map meets the filter; the programmer has written erronous parameters");
		}
    
    response.EnsureSuccessStatusCode();
    
    var requestUri = GetRequestUriFromResponse(response);

    if (requestUri is null)
    {
        Console.Error.WriteLine("RequestUri of /trackrandom is null");
        return null;
    }

    var trackId = GetTrackIdFromUri(requestUri);

    return trackId;
  }
}


## Sélection pseudo-alléatoire de circuits

Pour sélectionner un circuit alléatoire, nous choisissons une date alléatoire entre la date de publication du plus ancient et du plus nouveau circuit.

In [92]:

public class RandomMap()
{ 
  public static async Task<string> selectMapId() {
    string chosenId = await TMXAPI.getRandomMapID(new TMXInputs.MapInputsParameters{});

    Console.WriteLine(chosenId);

    return chosenId;
  }
}

string id = await RandomMap.selectMapId();

Head: trackrandom?taginclusive=True&primarytype=0&authortimemin=30000&authortimemax=120000&routes=0&inhasrecord=10&inenvmix=0&inunlimiter=0
2564751
